In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
import glob
import torch
from torch.utils.data import Dataset
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, f1_score

In [3]:
class RowwiseDataset(Dataset):
    def __init__(self, X_folder, y_attack_folder):
        self.X_data = pd.concat(
            [pd.read_parquet(f) for f in sorted(glob.glob(os.path.join(X_folder, "*.parquet")))]
        ).values
        self.y_attack_data = pd.concat(
            [pd.read_parquet(f)["Attack"] for f in sorted(glob.glob(os.path.join(y_attack_folder, "*.parquet")))]
        ).values
        assert len(self.X_data) == len(self.y_attack_data), \
            "Mismatched row counts between features and labels."
        assert self.y_attack_data.ndim == 1, "Target data should be 1D after column selection."

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X_data[idx], dtype=torch.float32),
            torch.tensor(self.y_attack_data[idx], dtype=torch.float32),
        )

In [4]:
X_train_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/X_train"
y_train_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/y_train_attack"

X_valid_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/X_valid"
y_valid_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/y_valid_attack"

X_test_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/X_test"
y_test_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-scaled-split/rmv_outlier_std_801010/y_test_attack"

In [5]:
train_dataset = RowwiseDataset(X_train_folder, y_train_attack_folder)
valid_dataset = RowwiseDataset(X_valid_folder, y_valid_attack_folder)
test_dataset = RowwiseDataset(X_test_folder, y_test_attack_folder)

X_train, y_train_attack = train_dataset.X_data, train_dataset.y_attack_data
X_valid, y_valid_attack = valid_dataset.X_data, valid_dataset.y_attack_data
X_test, y_test_attack = test_dataset.X_data, test_dataset.y_attack_data

y_train = y_train_attack
y_valid = y_valid_attack
y_test = y_test_attack

In [6]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (10025308, 48), y_train shape: (10025308,)
X_valid shape: (1430469, 48), y_valid shape: (1430469,)
X_test shape: (1270467, 48), y_test shape: (1270467,)


In [7]:
sizes = [8, 16, 32, 64]
results = []

for size in sizes:
    print(f"Training TabNet with n_a=n_d={size}")

    clf = TabNetClassifier(
        device_name='cuda',
        n_d=size,
        n_a=size,
        lambda_sparse=0,
        mask_type='entmax',
        optimizer_params=dict(lr=1e-2, weight_decay=1e-5),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        scheduler_params={
            "mode": "min",
            "factor": 0.5,
            "patience": 10,
            "min_lr": 1e-5,
        },
        verbose=1,
        seed=42
    )

    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric=["accuracy", "balanced_accuracy"],
        max_epochs=100000,
        patience=30,
        batch_size=1024 * 10,
        virtual_batch_size=128 * 10,
        compute_importance=False,
    )

    model_path = f"tabnet_model_size_{size}.zip"
    clf.save_model(model_path)

    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    predictions_path = f"cross_entropy_y_pred_size_{size}.csv"
    pd.DataFrame(y_pred, columns=["y_pred"]).to_csv(predictions_path, index=False)

    results.append({
        "size": size,
        "accuracy": acc,
        "f1_score": f1,
        "model_path": model_path,
        "predictions_path": predictions_path,
    })

results_df = pd.DataFrame(results)
results_df.to_csv("cross_entropy_results_summary.csv", index=False)

print("Tuning completed. Models, predictions, and results have been saved.")

Training TabNet with n_a=n_d=8


/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.18876 | val_0_accuracy: 0.75632 | val_0_balanced_accuracy: 0.54105 |  0:02:02s
epoch 1  | loss: 0.08314 | val_0_accuracy: 0.86161 | val_0_balanced_accuracy: 0.58908 |  0:04:03s
epoch 2  | loss: 0.07321 | val_0_accuracy: 0.96271 | val_0_balanced_accuracy: 0.61607 |  0:06:04s
epoch 3  | loss: 0.07068 | val_0_accuracy: 0.95037 | val_0_balanced_accuracy: 0.5921  |  0:08:05s
epoch 4  | loss: 0.06789 | val_0_accuracy: 0.95981 | val_0_balanced_accuracy: 0.60933 |  0:10:06s
epoch 5  | loss: 0.06528 | val_0_accuracy: 0.97109 | val_0_balanced_accuracy: 0.63169 |  0:12:07s
epoch 6  | loss: 0.06357 | val_0_accuracy: 0.79038 | val_0_balanced_accuracy: 0.59452 |  0:14:08s
epoch 7  | loss: 0.06735 | val_0_accuracy: 0.96929 | val_0_balanced_accuracy: 0.62949 |  0:16:09s
epoch 8  | loss: 0.06071 | val_0_accuracy: 0.97091 | val_0_balanced_accuracy: 0.60749 |  0:18:10s
epoch 9  | loss: 0.0564  | val_0_accuracy: 0.97433 | val_0_balanced_accuracy: 0.61406 |  0:20:11s
epoch 10 | loss: 0.0

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Training TabNet with n_a=n_d=16


...

Training TabNet with n_a=n_d=64


/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.10831 | val_0_accuracy: 0.96354 | val_0_balanced_accuracy: 0.67562 |  0:02:06s
epoch 1  | loss: 0.06786 | val_0_accuracy: 0.97489 | val_0_balanced_accuracy: 0.66512 |  0:04:13s
epoch 2  | loss: 0.06337 | val_0_accuracy: 0.97166 | val_0_balanced_accuracy: 0.68615 |  0:06:19s
epoch 3  | loss: 0.05893 | val_0_accuracy: 0.97379 | val_0_balanced_accuracy: 0.65417 |  0:08:26s
epoch 4  | loss: 0.05757 | val_0_accuracy: 0.97563 | val_0_balanced_accuracy: 0.66964 |  0:10:33s
epoch 5  | loss: 0.05674 | val_0_accuracy: 0.97594 | val_0_balanced_accuracy: 0.70471 |  0:12:40s
epoch 6  | loss: 0.05168 | val_0_accuracy: 0.97525 | val_0_balanced_accuracy: 0.68574 |  0:14:47s
epoch 7  | loss: 0.05035 | val_0_accuracy: 0.97013 | val_0_balanced_accuracy: 0.6327  |  0:16:53s
epoch 8  | loss: 0.05003 | val_0_accuracy: 0.97727 | val_0_balanced_accuracy: 0.70361 |  0:19:01s
epoch 9  | loss: 0.04908 | val_0_accuracy: 0.97925 | val_0_balanced_accuracy: 0.70726 |  0:21:08s
epoch 10 | loss: 0.0

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Tuning completed. Models, predictions, and results have been saved.
